In [9]:
##################  generate training and testing set ###############
import pandas as pd
import numpy as np
from datetime import date

In [24]:
def get_label(s):
    s = s.split(':')
    if s[0]=='nan':
        return 0
    elif (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8]))-date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days<=15:
        return 1
    else:
        return -1

In [11]:
off_train = pd.read_csv('/content/drive/MyDrive/5012project/ccf_offline_stage1_train.csv',header = None, low_memory= False)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
#off_train.head()
#2050 coupon_id. date_received:20160701~20160731, 76309 users(76307 in trainset, 35965 in online_trainset), 1559 merchants(1558 in trainset)

off_test = pd.read_csv('/content/drive/MyDrive/5012project/ccf_offline_stage1_test_revised.csv',low_memory=False)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']
#off_train.head()
#11429826 record(872357 with coupon_id),762858 user(267448 in off_train)

on_train = pd.read_csv('/content/drive/MyDrive/5012project/ccf_online_stage1_train.csv',header= None,low_memory=False)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']
#off_train.head()

In [12]:
dataset3 = off_test
feature3 = off_train[((off_train.date>='20160315')&(off_train.date<='20160630'))|((off_train.date=='null')&(off_train.date_received>='20160315')&(off_train.date_received<='20160630'))]
dataset2 = off_train[(off_train.date_received>='20160515')&(off_train.date_received<='20160615')]
feature2 = off_train[(off_train.date>='20160201')&(off_train.date<='20160514')|((off_train.date=='null')&(off_train.date_received>='20160201')&(off_train.date_received<='20160514'))]
dataset1 = off_train[(off_train.date_received>='20160414')&(off_train.date_received<='20160514')]
feature1 = off_train[(off_train.date>='20160101')&(off_train.date<='20160413')|((off_train.date=='null')&(off_train.date_received>='20160101')&(off_train.date_received<='20160413'))]

In [14]:
coupon3 = pd.read_csv('/content/drive/MyDrive/5012project/coupon3_feature.csv')
merchant3 = pd.read_csv('/content/drive/MyDrive/5012project/merchant3_feature.csv')
user3 = pd.read_csv('/content/drive/MyDrive/5012project/user3_feature.csv')
user_merchant3 = pd.read_csv('/content/drive/MyDrive/5012project/user_merchant3.csv')
other_feature3 = pd.read_csv('/content/drive/MyDrive/5012project/other_feature3.csv')
dataset3 = pd.merge(coupon3,merchant3,on='merchant_id',how='left')
dataset3 = pd.merge(dataset3,user3,on='user_id',how='left')
dataset3 = pd.merge(dataset3,user_merchant3,on=['user_id','merchant_id'],how='left')
dataset3 = pd.merge(dataset3,other_feature3,on=['user_id','coupon_id','date_received'],how='left')
dataset3.drop_duplicates(inplace=True)
print(dataset3.shape)

(222261, 52)


In [15]:
dataset3.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,4129537,450,9983,0.833333,1.0,20160712,2,12,12,30.0,5.0,1,10.0,475.0,90.0,2612.0,0.0,10.0,0.894118,0.0,0.034456,0.189474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1
1,4129537,450,9983,0.833333,1.0,20160712,2,12,12,30.0,5.0,1,10.0,7282.0,1149.0,33321.0,0.0,10.0,0.894064,0.0,0.034483,0.157786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1
2,6949378,1300,3429,0.833333,NaN,20160706,3,6,6,30.0,5.0,1,2.0,3.0,0.0,5.0,NaN,NaN,NaN,NaN,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0,3.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1
3,6949378,1300,3429,0.833333,NaN,20160706,3,6,6,30.0,5.0,1,2.0,116.0,17.0,95.0,0.0,8.0,0.666667,0.0,0.178947,0.146552,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0,3.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1
4,2166529,7113,6928,0.900000,5.0,20160727,3,27,27,200.0,20.0,1,1.0,155.0,3.0,252.0,0.0,0.0,0.000000,0.0,0.011905,0.019355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,2,1,-1,-1


In [16]:
dataset3.user_merchant_buy_total = dataset3.user_merchant_buy_total.replace(np.nan,0)
dataset3.user_merchant_any = dataset3.user_merchant_any.replace(np.nan,0)
dataset3.user_merchant_received = dataset3.user_merchant_received.replace(np.nan,0)
dataset3['is_weekend'] = dataset3.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
weekday_dummies = pd.get_dummies(dataset3.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset3 = pd.concat([dataset3,weekday_dummies],axis=1)
dataset3.drop(['merchant_id','day_of_week','coupon_count'],axis=1,inplace=True)
dataset3 = dataset3.replace('null',np.nan)

In [36]:
dataset3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222261 entries, 0 to 272905
Data columns (total 57 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   user_id                                       222261 non-null  int64  
 1   coupon_id                                     222261 non-null  int64  
 2   discount_rate                                 222261 non-null  float64
 3   distance                                      198763 non-null  float64
 4   date_received                                 222261 non-null  int64  
 5   day_of_month                                  222261 non-null  int64  
 6   days_distance                                 222261 non-null  int64  
 7   discount_man                                  217257 non-null  float64
 8   discount_jian                                 217257 non-null  float64
 9   is_man_jian                                   22

In [19]:
dataset3.to_csv('/content/drive/MyDrive/5012project/dataset3.csv',index=None)

In [30]:
coupon2 = pd.read_csv('/content/drive/MyDrive/5012project/coupon2_feature.csv')
merchant2 = pd.read_csv('/content/drive/MyDrive/5012project/merchant2_feature.csv')
user2 = pd.read_csv('/content/drive/MyDrive/5012project/user2_feature.csv')
user_merchant2 = pd.read_csv('/content/drive/MyDrive/5012project/user_merchant2.csv')
other_feature2 = pd.read_csv('/content/drive/MyDrive/5012project/other_feature2.csv')
dataset2 = pd.merge(coupon2,merchant2,on='merchant_id',how='left')
dataset2 = pd.merge(dataset2,user2,on='user_id',how='left')
dataset2 = pd.merge(dataset2,user_merchant2,on=['user_id','merchant_id'],how='left')
dataset2 = pd.merge(dataset2,other_feature2,on=['user_id','coupon_id','date_received'],how='left')
dataset2.drop_duplicates(inplace=True)
print(dataset2.shape)

(492696, 53)


In [31]:
dataset2.user_merchant_buy_total = dataset2.user_merchant_buy_total.replace(np.nan,0)
dataset2.user_merchant_any = dataset2.user_merchant_any.replace(np.nan,0)
dataset2.user_merchant_received = dataset2.user_merchant_received.replace(np.nan,0)
dataset2['is_weekend'] = dataset2.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
weekday_dummies = pd.get_dummies(dataset2.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset2 = pd.concat([dataset2,weekday_dummies],axis=1)
dataset2['label'] = dataset2.date.astype('str') + ':' +  dataset2.date_received.astype('str')

In [35]:
#dataset2['label'].head()
#dataset2.head()
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492696 entries, 0 to 520598
Data columns (total 56 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   user_id                                       492696 non-null  int64  
 1   discount_rate                                 492696 non-null  float64
 2   distance                                      437588 non-null  float64
 3   day_of_month                                  492696 non-null  int64  
 4   days_distance                                 492696 non-null  int64  
 5   discount_man                                  484630 non-null  float64
 6   discount_jian                                 484630 non-null  float64
 7   is_man_jian                                   492696 non-null  int64  
 8   total_sales                                   486585 non-null  float64
 9   sales_use_coupon                              48

In [33]:
dataset2.label = dataset2.label.apply(get_label)
dataset2.drop(['merchant_id','day_of_week','date','date_received','coupon_id','coupon_count'],axis=1,inplace=True)
dataset2 = dataset2.replace('null',np.nan)
#dataset2.to_csv('data/dataset2.csv',index=None)

In [39]:
dataset2.head()
dataset2.to_csv('/content/drive/MyDrive/5012project/dataset2.csv',index=None)

In [45]:
coupon1 = pd.read_csv('/content/drive/MyDrive/5012project/coupon1_feature.csv')
merchant1 = pd.read_csv('/content/drive/MyDrive/5012project/merchant1_feature.csv')
user1 = pd.read_csv('/content/drive/MyDrive/5012project/user1_feature.csv')
user_merchant1 = pd.read_csv('/content/drive/MyDrive/5012project/user_merchant1.csv')
other_feature1 = pd.read_csv('/content/drive/MyDrive/5012project/other_feature1.csv')
dataset1 = pd.merge(coupon1,merchant1,on='merchant_id',how='left')
dataset1 = pd.merge(dataset1,user1,on='user_id',how='left')
dataset1 = pd.merge(dataset1,user_merchant1,on=['user_id','merchant_id'],how='left')
dataset1 = pd.merge(dataset1,other_feature1,on=['user_id','coupon_id','date_received'],how='left')
dataset1.drop_duplicates(inplace=True)
print(dataset1.shape)

(255224, 53)


In [48]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255224 entries, 0 to 275258
Data columns (total 56 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   user_id                                       255224 non-null  int64  
 1   discount_rate                                 255224 non-null  float64
 2   distance                                      228622 non-null  float64
 3   day_of_month                                  255224 non-null  int64  
 4   days_distance                                 255224 non-null  int64  
 5   discount_man                                  247355 non-null  float64
 6   discount_jian                                 247355 non-null  float64
 7   is_man_jian                                   255224 non-null  int64  
 8   total_sales                                   250076 non-null  float64
 9   sales_use_coupon                              25

In [47]:
dataset1.user_merchant_buy_total = dataset1.user_merchant_buy_total.replace(np.nan,0)
dataset1.user_merchant_any = dataset1.user_merchant_any.replace(np.nan,0)
dataset1.user_merchant_received = dataset1.user_merchant_received.replace(np.nan,0)
dataset1['is_weekend'] = dataset1.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
weekday_dummies = pd.get_dummies(dataset1.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset1 = pd.concat([dataset1,weekday_dummies],axis=1)
dataset1['label'] = dataset1.date.astype('str') + ':' +  dataset1.date_received.astype('str')
dataset1.label = dataset1.label.apply(get_label)
dataset1.drop(['merchant_id','day_of_week','date','date_received','coupon_id','coupon_count'],axis=1,inplace=True)
dataset1 = dataset1.replace('null',np.nan)

In [49]:
dataset1.to_csv('/content/drive/MyDrive/5012project/dataset1.csv',index=None)